In [6]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
import wandb
from pathlib import Path

# 본 과제 제출자는 현재 우분투 도커 환경에서 작업중이므로 다음과 같이 경로 설정
BASE_PATH="/home/Deep-Learning-study"
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd()
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")

if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [7]:
from _01_code._08_fcn_best_practice.c_trainer import ClassificationTrainer
from _01_code._15_lstm_and_its_application.f_arg_parser import get_parser
from _01_code._15_lstm_and_its_application.g_crypto_currency_regression_train_lstm import get_btc_krw_data

In [8]:
def get_model():
    class MyModel(nn.Module):
        def __init__(self, n_input, n_output):
            super().__init__()

            # LSTM 레이어 정의
            self.lstm = nn.LSTM(
                input_size=n_input,      # 입력 특성의 수 (OHLCV: 5개)
                hidden_size=256,         # LSTM의 은닉 상태 크기
                num_layers=3,            # LSTM 레이어의 수 (3개의 층으로 구성)
                batch_first=True         # 입력 텐서의 형태를 (batch, seq, feature)로 지정
            )
            
            # 완전연결 레이어 정의
            self.fcn = nn.Linear(
                in_features=256,         # LSTM의 hidden_size와 동일
                out_features=n_output    # 출력 클래스 수 (2: 상승/하락)
            )

        def forward(self, x):
            # LSTM 레이어 통과
            # x 입력 shape: (batch_size, sequence_length, n_input)
            # hidden: (h_n, c_n) - LSTM의 최종 은닉 상태와 셀 상태
            x, hidden = self.lstm(x)
            
            # 마지막 시퀀스의 출력만 선택
            # x shape: (batch_size, sequence_length, hidden_size)
            # x[:, -1, :] shape: (batch_size, hidden_size)
            x = x[:, -1, :]
            
            # 선형 레이어를 통과시켜 클래스 점수 생성
            # 출력 shape: (batch_size, n_output)
            x = self.fcn(x)
            return x  # CrossEntropyLoss를 사용할 것이므로 softmax는 여기서 적용하지 않음

    # 모델 인스턴스 생성
    my_model = MyModel(
        n_input=5,    # 입력 특성 수 (OHLCV)
        n_output=2    # 출력 클래스 수 (상승/하락)
    )

    return my_model

In [9]:
def main(args):
    # 현재 시간을 문자열로 변환 (실행 식별자로 사용)
    run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # wandb 설정을 위한 하이퍼파라미터 딕셔너리 생성
    config = {
        'epochs': args.epochs,                    # 총 학습 에폭 수
        'batch_size': args.batch_size,           # 미니배치 크기
        'validation_intervals': args.validation_intervals,  # 검증 수행 주기
        'learning_rate': args.learning_rate,      # 학습률
        'early_stop_patience': args.early_stop_patience,  # 조기 종료 인내 횟수
        'early_stop_delta': args.early_stop_delta,  # 조기 종료 임계값
        'weight_decay': args.weight_decay         # L2 정규화 강도
    }

    # wandb 프로젝트 초기화 (분류 태스크용)
    project_name = "lstm_classification_btc_krw"
    wandb.init(
        mode="online" if args.wandb else "disabled",  # wandb 활성화 여부
        project=project_name,                         # 프로젝트 이름
        notes="btc_krw experiment with lstm",         # 실험 설명
        tags=["lstm", "classification", "btc_krw"],   # 관련 태그 (분류 태스크 명시)
        name=run_time_str,                           # 실행 이름 (시간 기반)
        config=config                                # 설정값들
    )
    # 설정값 출력
    print(args)
    print(wandb.config)

    # 데이터 로더 생성 (분류 태스크용 데이터 준비)
    train_data_loader, validation_data_loader, _ = get_btc_krw_data(is_regression=False)
    
    # GPU 사용 가능 여부 확인 및 디바이스 설정
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Training on device {device}.")

    # 분류 모델 인스턴스 생성 및 지정된 디바이스로 이동
    model = get_model()
    model.to(device)

    # Adam 옵티마이저 설정
    optimizer = optim.Adam(
        model.parameters(),
        lr=wandb.config.learning_rate,      # 학습률
        weight_decay=wandb.config.weight_decay  # L2 정규화
    )

    # 분류 학습 객체 생성
    classification_trainer = ClassificationTrainer(
        project_name=project_name,           # 프로젝트 이름
        model=model,                         # 학습할 모델
        optimizer=optimizer,                 # 옵티마이저
        train_data_loader=train_data_loader,  # 학습 데이터 로더
        validation_data_loader=validation_data_loader,  # 검증 데이터 로더
        test_data_loader=None,              # 테스트 데이터 로더 (사용 안 함)
        run_time_str=run_time_str,          # 실행 식별자
        wandb=wandb,                        # wandb 객체
        device=device,                      # 학습 디바이스
        checkpoint_file_path=CHECKPOINT_FILE_PATH  # 모델 저장 경로
    )
    
    # 학습 실행
    classification_trainer.train_loop()

    # wandb 실행 종료
    wandb.finish()

In [12]:
if __name__ == "__main__":
    import sys
    if 'ipykernel' in sys.modules:  # Jupyter Notebook에서 실행 중인지 확인
        # Jupyter에서 실행할 때는 기본값 사용
        class Args:
            def __init__(self):
                self.wandb = True
                self.batch_size = 64
                self.epochs = 200
                self.learning_rate = 0.0001
                self.weight_decay = 0.00001
                self.validation_intervals = 1
                self.early_stop_patience = 10
                self.early_stop_delta = 0.001
        args = Args()
    else:
        # 일반 Python 스크립트로 실행할 때는 argparse 사용
        parser = get_parser()
        args = parser.parse_args()
    
    main(args)

{'epochs': 200, 'batch_size': 64, 'validation_intervals': 1, 'learning_rate': 0.0001, 'early_stop_patience': 10, 'early_stop_delta': 0.001, 'weight_decay': 1e-05}
Training on device cuda:0.
[Epoch   1] T_loss: 0.69205, T_accuracy: 51.8293 | V_loss: 0.69065, V_accuracy: 55.0000 | Early stopping is stated! | T_time: 00:00:00, T_speed: 0.000
[Epoch   2] T_loss: 0.69105, T_accuracy: 53.6863 | V_loss: 0.69643, V_accuracy: 44.0000 | Early stopping counter: 1 out of 10 | T_time: 00:00:00, T_speed: 0.000
[Epoch   3] T_loss: 0.69078, T_accuracy: 53.2982 | V_loss: 0.69525, V_accuracy: 44.0000 | Early stopping counter: 2 out of 10 | T_time: 00:00:00, T_speed: 0.000
[Epoch   4] T_loss: 0.69145, T_accuracy: 53.2151 | V_loss: 0.69714, V_accuracy: 44.0000 | Early stopping counter: 3 out of 10 | T_time: 00:00:00, T_speed: 0.000
[Epoch   5] T_loss: 0.69120, T_accuracy: 53.0488 | V_loss: 0.69689, V_accuracy: 44.0000 | Early stopping counter: 4 out of 10 | T_time: 00:00:00, T_speed: 0.000
[Epoch   6] T_l

Epoch,▁▂▂▃▄▅▅▆▇▇█
Training accuracy (%),▁█▇▆▆▇▄▆▇▆▆
Training loss,█▃▁▅▃▂▃▂▂▂▂
Training speed (epochs/sec.),▁▁▁▁▁▅▅▆▇▇█
Validation accuracy (%),█▁▁▁▁▁▁▁▁▁▁
Validation loss,▁▆▅▆▆█▅▇▆▅▅
Epoch,11
Training accuracy (%),53.0765
Training loss,0.69093
Training speed (epochs/sec.),11
Validation accuracy (%),44
